# Catapult codesign

Model: `(13,21,2), n_filters=5, pool_size=3`

Disable some console warnings on the ASIC-group servers

In [1]:
import os
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [2]:
import hls4ml

import numpy as np
import yaml
import json
# from tensorflow.keras.models import Model
from models import CreateModel

from loss import *
from qkeras import quantized_bits


## Set parameters

In [3]:
# You can define a JSON configuration file locally
# {
#    "data_base_dir": "/data/dajiang/smartPixels",
#    "tfrecords_base_dir" : "/data/dajiang/smartPixels",
#    "model_base_dir": "/home/dajiang/smart-pixels-ml/weights"
# }
config_file_path = "config.json"

# If the file does not exist, the notebook uses default values for those entries
data_base_dir = "/data/dajiang/smartPixels/dataset_2s"
tfrecords_base_dir = "/data/dajiang/smartPixels/tfrecords"
model_base_dir = "/home/dajiang/smart-pixels-ml/weights"

if os.path.exists(config_file_path):
    with open(config_file_path, "r") as file:
        data = json.load(file)
        data_base_dir = data.get("data_base_dir")
        tfrecords_base_dir = data.get("tfrecords_base_dir")
        model_base_dir = data.get("model_base_dir")
    print(f"Use config info from file: {data_base_dir}, {tfrecords_base_dir}, {model_base_dir}")
else:
    print(f"File does not exist. Use default config info: {data_base_dir}, {tfrecords_base_dir}, {model_base_dir}")

Use config info from file: /home/giuseppe/research/projects/smartpixels/data/dataset_2s, /home/giuseppe/research/projects/smartpixels/davidgjiang-smart-pixels-ml/tfrecords, /home/giuseppe/research/projects/smartpixels/davidgjiang-smart-pixels-ml/weights


In [4]:
#HDF5_WEIGHTS_FILE = f"3Dallparamstiny-fastml-avgqbits-noint-4bits-8bitavg-2279bbff-checkpoints/weights.899-t-23196.44-v-25047.58.hdf5"
best_model_weights_hdf5 = f"{model_base_dir}/weights_7pitches/best_model_weights.hdf5"
best_model_architecture_json = f"{model_base_dir}/weights_7pitches/best_model_architecture.json"
best_model_hdf5 = f"{model_base_dir}/weights_7pitches/best_model.hdf5"

# TODO: Set the right precision
FXD_W = 4 # Fixed-point precision, word bit width
FXD_I = 1 # Fixed-point precision, integer-part bit width

## Load input data

### Toy data

In [5]:
# TODO: Get rid of the toy data, and use actual data

# Set a seed to have the same input traces on every run
np.random.seed(42)

B = 1
H = 13
W = 21
C = 2

toy_data = np.random.rand(B,
                          H,
                          W,
                          C)

def data_generator(left, right, step):
    next_value = left
    
    def function():
        nonlocal next_value 
        current_value = next_value
        next_value = next_value + 0.125
        if next_value > right:
            next_value = -1
        return current_value
        
    return function

# Generate data between [-1, 0.875] step 0.125
get_next_value = data_generator(-1, 0.875, 0.125)
for i in range(B):
    for h in range(H):
        for w in range(W):
            for d in range(C):
                toy_data[i][h][w][d] = get_next_value()

#print(toy_data.shape)
#print(toy_data)

In [6]:
# Quantize input data
q_toy_data = quantized_bits(FXD_W, FXD_I-1, alpha=1)(toy_data).numpy()

#print(q_toy_data.shape)
#print(q_toy_data)

### Real data

## QKeras model

### Load QKeras model

In [7]:
# Load the whole model from HDF5 file
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

co = {"custom_loss": custom_loss}
_add_supported_quantized_objects(co)
model = load_model(best_model_hdf5, custom_objects=co)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d (QSepar  (None, 11, 19, 5)         33        
 ableConv2D)                                                     
                                                                 
 batch_normalization (Batch  (None, 11, 19, 5)         20        
 Normalization)                                                  
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 11, 19, 5)         30        
                                                                 
 batch_normalization_1 (Bat  (None, 11, 19, 5)         20    

In [ ]:
# TODO: This gives an error:
# ValueError: Layer count mismatch when loading weights from file. Model expected 5 layers, found 9 saved layers.
#
# # Load model weights from HDF5 file, while recreate architecture from scripts
# model = CreateModel((13,21,2), n_filters=5, pool_size=3)
# model.load_weights(best_model_weights_hdf5)
# model.summary()

### Slice QKeras model

In [ ]:
# SLICE_TO_LAYER_NUM = 12
# print("Layer count: {}/{}".format(SLICE_TO_LAYER_NUM, len(model.layers)-1))
# assert(SLICE_TO_LAYER_NUM < len(model.layers))
# model = Model(inputs=model.input, 
#                   outputs=model.layers[SLICE_TO_LAYER_NUM].output)

In [ ]:
# model.summary()

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=True)

### Run QKeras model

#### Prediction

In [ ]:
# TODO: Use real data

y_qkeras = model.predict(np.ascontiguousarray(q_toy_data))

#### Profiling

In [ ]:
# TODO: Use real data

qkeras_trace = hls4ml.model.profiling.get_ymodel_keras(model, q_toy_data)

#### Save .dat files

In [ ]:
# TODO: Use real data

# Save input features and model predictions just the top 20
np.savetxt("tb_input_features.dat", q_toy_data.reshape(B, -1), fmt="%f")
np.savetxt("tb_output_predictions.dat", y_qkeras, fmt="%f")
#np.savetxt("y_test_labels.dat", y_test, fmt="%d")

## hls4ml model

### Configure hls4ml model

In [ ]:
config_ccs = hls4ml.utils.config.create_config(
    backend = "Catapult",
    project_name = "myproject",
    output_dir = "myproject_hls4ml_prj",
    tech = "asic",
    asiclibs = "saed32rvt_tt0p78v125c_beh",
    asicfifo = "hls4ml_lib.mgc_pipe_mem",
    clock_period = 10,
    io_type = "io_parallel",
    csim=0, SCVerify=0, Synth=1
)
#print(config_ccs)

In [ ]:
config_ccs["HLSConfig"] = hls4ml.utils.config_from_keras_model(
    model,
    granularity="name",
    default_precision="ac_fixed<16,6,true>",
    default_reuse_factor=1
)

# Point to the model definition, weights/biase values and C++ testbench data files
config_ccs["KerasH5"] = best_model_weights_hdf5
config_ccs["KerasJson"] = best_model_architecture_json
config_ccs["InputData"] = "tb_input_features.dat"
config_ccs["OutputPredictions"] = "tb_output_predictions.dat"

In [ ]:
# TODO: is this necessary?

with open("myproject_config.yml", "w") as yaml_file:
    yaml.dump(config_ccs, yaml_file, explicit_start=False, default_flow_style=False)

In [ ]:
# Enable tracing for all of the layers
for layer in config_ccs["HLSConfig"]["LayerName"].keys():
    print("Enable tracing for layer:", layer)
    config_ccs["HLSConfig"]["LayerName"][layer]["Trace"] = True

In [ ]:
# Convert QKeras model to Catapult HLS C++
hls_model_ccs = hls4ml.converters.keras_to_hls(config_ccs)

In [ ]:
#hls4ml.utils.plot_model(hls_model_ccs, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
# Writing HLS project
hls_model_ccs.compile()

### Run hls4ml model

#### Prediction

In [ ]:
# TODO: Use real data

y_ccs = hls_model_ccs.predict(np.ascontiguousarray(q_toy_data))

#### Profiling

In [ ]:
# TODO: Use real data

# Run tracing on the test set for the hls4ml model (fixed-point precision) 
pred_ccs, trace_ccs = hls_model_ccs.trace(q_toy_data)

## Compare QKeras and hls4ml

#### Trace visual inspection

In [ ]:
# Print the traces on console
N_ELEMENTS=10

# Backup print options
bkp_threshold = np.get_printoptions()["threshold"]
bkp_linewidth = np.get_printoptions()["linewidth"]

# Set print options
np.set_printoptions(threshold=np.inf, linewidth=np.inf)

print("input", toy_data[0][0][0][:N_ELEMENTS])
for key in trace_ccs.keys():
    print("-------")
    print(key, trace_ccs[key].shape)
    print("[keras] ", key, qkeras_trace[key].flatten()[:N_ELEMENTS])
    #print(mse(hls4ml_trace[key][0].flatten(), qkeras_trace[key][0].flatten()))
    print("[hls4ml]", key, trace_ccs[key].flatten()[:N_ELEMENTS])
    #print(key, qkeras_trace[key].shape)
    
# Restore print options
np.set_printoptions(threshold=bkp_threshold, linewidth=bkp_linewidth)

#### MSE per layer

In [ ]:
def mse(actual, predicted):
    return ((actual - predicted) ** 2).mean()

for key in trace_ccs.keys():
    print("-------")
    print("MSE {} {}".format(key, mse(trace_ccs[key][0].flatten(), qkeras_trace[key].flatten())))

#### Correlation plots

In [ ]:
import matplotlib.pyplot as plt
import math

# Evaluate correlation plots
for layer in trace_ccs.keys():
    if "_alpha" in layer:
        continue
    plt.figure()
    klayer = layer
    if "_linear" in layer:
        klayer = layer.replace("_linear", "")
    min_x = min(np.amin(trace_ccs[layer]), np.amin(qkeras_trace[klayer]))
    max_x = max(np.amax(trace_ccs[layer]), np.amax(qkeras_trace[klayer]))
    golden_min_x = np.amin(qkeras_trace[klayer])
    golden_max_x = np.amax(qkeras_trace[klayer])
    #print("{}, range [{}, {}], range {}, bits {}".format(layer, golden_min_x, golden_max_x, abs(golden_min_x) + abs(golden_max_x), math.ceil(math.log2(abs(golden_min_x) + abs(golden_max_x)))))
    plt.plot([min_x, max_x], [min_x, max_x], c="gray")
    plt.scatter(trace_ccs[layer].flatten(), qkeras_trace[klayer].flatten(), s=0.2, c="red")
    plt.xlabel("hls4ml {}".format(layer))
    plt.ylabel("QKeras {}".format(klayer))


## Model synthesis

In [ ]:
%%time

# TODO: Check the parameters for the build function (Catapult)

#report = hls_model.build(csim=True, synth=False, cosim=False, validation=False, export=False, vsynth=False, reset=False)
report = hls_model_ccs.build(csim=True, synth=True, cosim=False, validation=False, export=False, vsynth=False, bup=False)

### Results